# Advanced SQL Querying

**Purpose**: Complex queries and joins
**Data**: 2022-2023

Learn raw SQL for advanced filtering.

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent / 'src'))

from pymaude import MaudeDatabase
import pandas as pd

In [2]:
db = MaudeDatabase('getting_started.db', verbose=True)
db.add_years(years='2022-2023', tables=['device', 'master', 'patient'], download=False, data_dir='./maude_data', interactive=False)


Grouping years by file for optimization...

Processing data files...

device for year 2022 already loaded and unchanged, skipping

device for year 2023 already loaded and unchanged, skipping

master for years 2022-2023 already loaded and unchanged, skipping

patient for years 2022-2023 already loaded and unchanged, skipping

Creating indexes...

Database update complete


## Complex Filters

In [3]:
# Multiple device types with date filter - using patient outcomes for accurate death counts
sql = """
    SELECT m.*, d.GENERIC_NAME, d.BRAND_NAME, p.SEQUENCE_NUMBER_OUTCOME
    FROM master m
    JOIN device d ON m.MDR_REPORT_KEY = d.MDR_REPORT_KEY
    LEFT JOIN patient p ON m.MDR_REPORT_KEY = p.MDR_REPORT_KEY
    WHERE (d.GENERIC_NAME LIKE '%pacemaker%' OR d.GENERIC_NAME LIKE '%defibrillator%')
      AND m.DATE_RECEIVED >= '2023-01-01'
      AND p.SEQUENCE_NUMBER_OUTCOME LIKE '%D%'
"""
results = db.query(sql)
print(f'Found {len(results):,} fatal events')

Found 3,424 fatal events


## Aggregations

In [4]:
# Top manufacturers by event count
sql = """
    SELECT d.MANUFACTURER_D_NAME, COUNT(*) as events
    FROM device d
    WHERE d.GENERIC_NAME LIKE '%catheter%'
    GROUP BY d.MANUFACTURER_D_NAME
    ORDER BY events DESC
    LIMIT 10
"""
top_mfg = db.query(sql)
print(top_mfg)

                              MANUFACTURER_D_NAME  events
0                   BOSTON SCIENTIFIC CORPORATION   27915
1            C.R. BARD, INC. (COVINGTON) -1018233   18924
2              C.R. BARD, INC. (BASD) -3006260740   14500
3                            BIOSENSE WEBSTER INC    9063
4                                 ABBOTT VASCULAR    4330
5                                ST. JUDE MEDICAL    4248
6                         ARROW INTERNATIONAL LLC    4020
7  BECTON DICKINSON INFUSION THERAPY SYSTEMS INC.    3778
8                        ARROW INTERNATIONAL INC.    3735
9                                        COOK INC    3370


In [5]:
db.close()
print('See docs/api_reference.md for more!')

See docs/api_reference.md for more!
